# 2022年5月14日 - 第14个恋恋日献礼
### 东方歌词生成器+高性能平行翻译器 (Paralleled Touhou Lyric Generator and Translator, PaToL-GT)
#### 以及古明地恋主题曲（哈德曼的妖怪少女）歌词专用生成器

In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
# 锁住numpy和pytorch的随机数种子（保证复现性，在实用的时候可以不锁）
import numpy as np
import torch

seed=514
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

### 生成器和翻译器都是通过微调日文GPT2实现的
使用了**thbwiki**上的**7122**首东方曲训练了生成器，恋恋专用的生成器以此为基础使用了其中的**91**首+从**東方同人CDの歌詞＠wiki**额外收集的**68**首（可能有重复）原曲为**哈德曼的妖怪少女**的歌词训练。\
翻译器首先使用了**600k**条中日维基平行语料进行预训练，然后又在抽取到的**93k（thbwiki）+547k（萌娘百科）** 的日->中歌词语料上迁移后继续训练。
#### 依赖库：
numpy, pytorch, transformers (为了合并词典，必须最新版的**4.18.0**)\
rjieba (中文分词器)\
fugashi, sentencepiece (日文分词器)

In [3]:
# 导入预定义好的生成和翻译模型框架
from models import TouhouMusicTranslator, TouhouMusicGenerator

# 设置显卡加速（一般生成需要4GB的显存，使用Beam Search需要8GB），如果没有可用显卡也可以使用CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 东方歌词翻译器

In [4]:
# 为翻译器加载预训练好的状态
translator = TouhouMusicTranslator(f'./translator_anime_music_medium_state.pth', device=device)

### 和Google翻译器的性能比较

**评价指标: BERTScore** from [ICLR2020](https://openreview.net/forum?id=SkeHuCVFDr)\
**测试集**和**开发集**各由128句东方曲中歌词及其翻译组成。
    
| 翻译器 |  开发集 | 测试集 |
| :---------| :-----: | :------: |
| Google |  68.8   |   67.3   |
| PaToL  |  **73.8**   |   **71.6**   |

**翻译效率**
    
| 翻译模式 |  CPU | GPU |
| :---------| :-----: | :------: |
| Greedy |  2.539秒/句   |   0.088秒/句   |
| Beam Search (size=6)  |  4.116秒/句   |   0.095秒/句   |

**以下是一些样例**：

In [5]:
# 导入Google翻译器
from googletrans import Translator
google_translator = Translator(service_urls=['translate.google.cn'])

with open('./translate_cases.txt', encoding='utf-8') as fp:
    cases = [eval(line) for line in fp.read().strip().split('\n')]
    
for case in cases:
    translation = translator.translate(case['ja'])
    google_translation = google_translator.translate(case['ja'], dest='zh-CN').text
    print("日文原文：", case['ja']); print('.'*100)
    print("Google翻译：", google_translation)
    print("PaToL翻译：", translation)
    print("标准翻译：", case['zh']); print('-'*50)

日文原文： 閉じた恋の瞳
....................................................................................................
Google翻译： 闭合的爱眼睛
PaToL翻译： 紧闭的恋爱之瞳
标准翻译： 紧闭的恋之瞳
--------------------------------------------------
日文原文： 儚さ故 美しい
....................................................................................................
Google翻译： 优雅而美丽
PaToL翻译： 因虚幻而美丽
标准翻译： 因虚幻而美丽
--------------------------------------------------
日文原文： シアワセなんてものは
....................................................................................................
Google翻译： siawase
PaToL翻译： 幸福什么的
标准翻译： 幸福这种东西
--------------------------------------------------
日文原文： 枯れぬ花を咲かせたい 輝く銀の花束
....................................................................................................
Google翻译： 一束闪亮的银花束，想和枯萎的花朵开花
PaToL翻译： 想让那永不枯萎的花朵绽放闪耀的银花束
标准翻译： 多想花朵绽放永不枯萎 光辉闪耀的银色花束
--------------------------------------------------
日文原文： （荷取）: 希望　溢れる　(魔理沙：明日を行く)
.......................................................................

## 东方歌词生成器

In [6]:
# 加载东方歌词生成器的状态
generator = TouhouMusicGenerator(f'./gpt2_touhou_medium_state.pth', translator=translator, device=device)

In [7]:
prefix = "閉じた恋の瞳"
print("作曲：ZUN    作词：GPT2-Touhou")
print("-"*50)
lyrics = generator.generate(prefix, temperature=1.0, step_range=(32, 64), log=True)

作曲：ZUN    作词：GPT2-Touhou
--------------------------------------------------
歌词： 閉じた恋の瞳に惑う
翻译： 被封闭的恋爱之瞳所迷惑
--------------------------------------------------
歌词： 声を聞きたくて聳え立つ
翻译： 想要听到你的声音才是我的目标
--------------------------------------------------
歌词： 標識が消えれば 見えにくい暗がり
翻译： 看不清路标就会变得模糊的黑暗
--------------------------------------------------
歌词： まるで木々が芽吹く頃に咲く花のようだから
翻译： 就像在树木萌芽之时绽放的花朵一样
--------------------------------------------------
歌词： 君 どうかひとつ
翻译： 请让我一个
--------------------------------------------------
歌词： 叶えてよ一度でいい
翻译： 想实现一次就好
--------------------------------------------------
歌词： その想い響いて
翻译： 回响着这份思念
--------------------------------------------------


## 东方歌词生成器（恋恋专用版）

In [8]:
# 切换歌词生成器的状态为恋恋专用~
generator.load(f'./gpt2_koishi_medium_state.pth')

In [9]:
prefix = "閉じた恋の瞳"
print("作曲：ZUN    作词：GPT2-Koishi")
print("-"*50)
lyrics = generator.generate(prefix, temperature=1.0, step_range=(32, 64), log=True)

作曲：ZUN    作词：GPT2-Koishi
--------------------------------------------------
歌词： 閉じた恋の瞳で見るあこがれは
翻译： 用紧闭的爱之瞳所见的憧憬
--------------------------------------------------
歌词： 黒い薔薇の檻
翻译： 黑色蔷薇的牢笼
--------------------------------------------------
歌词： 重力の彼方へ
翻译： 向着重力的彼方
--------------------------------------------------
歌词： おとぎごっこ さて
翻译： 来玩捉迷藏吧
--------------------------------------------------
歌词： 今宵の野望は
翻译： 今夜的野心
--------------------------------------------------
歌词： 私はあなた
翻译： 我就是你
--------------------------------------------------
歌词： あなたが引き寄せる 誰かの想いを察して
翻译： 你所吸引的是谁的思念
--------------------------------------------------
歌词： with you inside and outside
翻译： 在你的身边
--------------------------------------------------


本项目为基于东方project二次创作以及动画歌曲开发的辅助工具，项目中所使用的歌词以及翻译内容的版权均属于对应的艺术创作者和百科社区贡献者，故请勿用于商业用途。\
将本工具用于东方project或者动画作品领域时，无需注明来源，若用于其它领域时，希望加以注明。对训练数据的使用需要遵循源版权者的规定。